In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
%cd /content/drive/My Drive/Colab Notebooks/iris_recognition

/content/drive/My Drive/Colab Notebooks/iris_recognition


In [0]:
from dataclasses import dataclass
from typing import Dict

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


# Directory with normalized photos
INPUT_DATA_DIR = 'normalized_splitted'

# Data augmentation for training and validation
DATA_TRANSFORMS = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


@dataclass
class TrainConfig:
    data: Dict[str, ImageFolder]
    loaders: Dict[str, DataLoader]

    def __post_init__(self):
        self.class_names = self.data['train'].classes
        self.data_sizes = {x: len(self.data[x]) for x in ['train', 'val']}
        self.device = torch.device("cuda:0" if torch.cuda.is_available()
                                   else "cpu")

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD
        self.lr_scheduler = lr_scheduler.StepLR

    def get_model(self) -> nn.Module:
        model = models.resnet50(pretrained=True)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, len(self.class_names))

        return model.to(self.device)


def create_train_config(batch_size: int = 4, shuffle: bool = True,
                        num_workers: int = 4) -> TrainConfig:
    image_datasets = {
        x: ImageFolder(os.path.join(INPUT_DATA_DIR, x), DATA_TRANSFORMS[x])
        for x in ['train', 'val']
    }

    data_loaders = {
        x: DataLoader(image_datasets[x], batch_size=batch_size,
                      shuffle=shuffle, num_workers=num_workers)
        for x in ['train', 'val']
    }

    return TrainConfig(image_datasets, data_loaders)


In [0]:
import torch
import torch.nn as nn
import time
import copy

CHECKPOINT_FILE_NAME = "iris_recognition_trained_model.pt"


def train_model(train_config: TrainConfig,
                model: nn.Module,
                criterion,
                optimizer,
                scheduler,
                num_epochs: int = 50):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    accuracies = {
        "train": [],
        "val": []
    }

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in train_config.loaders[phase]:
                inputs = inputs.to(train_config.device)
                labels = labels.to(train_config.device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / train_config.data_sizes[phase]
            epoch_acc = running_corrects.double() / train_config.data_sizes[
                phase]

            accuracies[phase].append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'
                  .format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'
          .format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, accuracies


In [38]:
train_config = create_train_config()
model = train_config.get_model()
optimizer = train_config.optimizer(model.parameters(),
                                    lr=0.001,
                                    momentum=0.9)
scheduler = train_config.lr_scheduler(optimizer, step_size=7, gamma=0.1)

trained_model, accuracies = train_model(
    train_config=train_config,
    model=model,
    optimizer=optimizer,
    criterion=train_config.criterion,
    scheduler=scheduler
)

torch.save({
    "model_state_dict": trained_model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "train_accuracies": accuracies["train"],
    "validation_accuracies": accuracies["val"]
}, CHECKPOINT_FILE_NAME)


Epoch 0/49
----------
train Loss: 3.5732 Acc: 0.0383
val Loss: 3.4644 Acc: 0.0833

Epoch 1/49
----------
train Loss: 3.3147 Acc: 0.0574
val Loss: 2.7479 Acc: 0.2667

Epoch 2/49
----------
train Loss: 3.1371 Acc: 0.1196
val Loss: 2.4714 Acc: 0.3167

Epoch 3/49
----------
train Loss: 2.8162 Acc: 0.2201
val Loss: 1.9515 Acc: 0.4833

Epoch 4/49
----------
train Loss: 2.4248 Acc: 0.2823
val Loss: 1.8909 Acc: 0.5167

Epoch 5/49
----------
train Loss: 2.3351 Acc: 0.3014
val Loss: 1.2817 Acc: 0.5667

Epoch 6/49
----------
train Loss: 2.2069 Acc: 0.3971
val Loss: 1.3819 Acc: 0.5833

Epoch 7/49
----------
train Loss: 1.9608 Acc: 0.4306
val Loss: 0.9026 Acc: 0.7667

Epoch 8/49
----------
train Loss: 1.6588 Acc: 0.5742
val Loss: 0.7968 Acc: 0.8333

Epoch 9/49
----------
train Loss: 1.6123 Acc: 0.5933
val Loss: 0.7200 Acc: 0.8167

Epoch 10/49
----------
train Loss: 1.6548 Acc: 0.5646
val Loss: 0.7710 Acc: 0.8167

Epoch 11/49
----------
train Loss: 1.5390 Acc: 0.6029
val Loss: 0.7157 Acc: 0.8000

Ep